# Introduction to midodrine examination

In [1]:
from google.colab import auth
auth.authenticate_user()
from google.cloud import bigquery

# Initialize client with project ID
client = bigquery.Client(project='sccm-datathon-2025-participant')

In [ ]:
query = f"""

WITH midodrine_meds AS (
  SELECT DISTINCT
    m.patientunitstayid,
    m.drugname,
    m.drugstartoffset,
    m.drugstopoffset,
    m.dosage,
    m.routeadmin
  FROM `sccm-discovery.eicu_crd_ii_v0_2_0.medication` m
  WHERE (LOWER(m.drugname) LIKE '%midodrine%'
         OR LOWER(m.drugname) LIKE '%proamatine%'
         OR LOWER(m.drugname) LIKE '%orvaten%')
      AND m.drugstartoffset IS NOT NULL
      AND LOWER(COALESCE(m.routeadmin, '')) IN ('po', 'oral', 'enteral', 'gt', 'ng', 'peg', '')
)
SELECT * FROM midodrine_meds;

"""
midodrine_admin = client.query(query).to_dataframe()
midodrine_admin

In [ ]:
query = f"""

WITH medication_vaso AS (
    SELECT DISTINCT
        m.patientunitstayid,
        m.drugname,
        m.drugstartoffset,
        m.drugstopoffset,
        CASE
            WHEN LOWER(m.drugname) LIKE '%norepinephrine%' OR LOWER(m.drugname) LIKE '%noradrenaline%' OR LOWER(m.drugname) LIKE '%levophed%' THEN 'norepinephrine'
            WHEN LOWER(m.drugname) LIKE '%epinephrine%' OR LOWER(m.drugname) LIKE '%adrenaline%' THEN 'epinephrine'
            WHEN LOWER(m.drugname) LIKE '%dopamine%' THEN 'dopamine'
            WHEN LOWER(m.drugname) LIKE '%vasopressin%' OR LOWER(m.drugname) LIKE '%pitressin%' THEN 'vasopressin'
            WHEN LOWER(m.drugname) LIKE '%phenylephrine%' OR LOWER(m.drugname) LIKE '%neosynephrine%' THEN 'phenylephrine'
            WHEN LOWER(m.drugname) LIKE '%dobutamine%' THEN 'dobutamine'
        END as vasopressor_type
    FROM `sccm-discovery.eicu_crd_ii_v0_2_0.medication` m
    WHERE (LOWER(m.drugname) LIKE '%norepinephrine%' OR LOWER(m.drugname) LIKE '%noradrenaline%' OR LOWER(m.drugname) LIKE '%levophed%'
           OR LOWER(m.drugname) LIKE '%epinephrine%' OR LOWER(m.drugname) LIKE '%adrenaline%'
           OR LOWER(m.drugname) LIKE '%dopamine%' OR LOWER(m.drugname) LIKE '%intropin%'
           OR LOWER(m.drugname) LIKE '%vasopressin%' OR LOWER(m.drugname) LIKE '%pitressin%'
           OR LOWER(m.drugname) LIKE '%phenylephrine%' OR LOWER(m.drugname) LIKE '%neosynephrine%'
           OR LOWER(m.drugname) LIKE '%dobutamine%' OR LOWER(m.drugname) LIKE '%dobutrex%')
        AND m.drugstartoffset IS NOT NULL
),
infusion_vaso AS (
    SELECT DISTINCT
        i.patientunitstayid,
        i.drugname,
        i.infusionoffset as drugstartoffset,
        LEAD(i.infusionoffset) OVER (PARTITION BY i.patientunitstayid, i.drugname ORDER BY i.infusionoffset) as drugstopoffset,
        CASE
            WHEN LOWER(i.drugname) LIKE '%norepinephrine%' OR LOWER(i.drugname) LIKE '%noradrenaline%' THEN 'norepinephrine'
            WHEN LOWER(i.drugname) LIKE '%epinephrine%' OR LOWER(i.drugname) LIKE '%adrenaline%' THEN 'epinephrine'
            WHEN LOWER(i.drugname) LIKE '%dopamine%' THEN 'dopamine'
            WHEN LOWER(i.drugname) LIKE '%vasopressin%' THEN 'vasopressin'
            WHEN LOWER(i.drugname) LIKE '%phenylephrine%' THEN 'phenylephrine'
            WHEN LOWER(i.drugname) LIKE '%dobutamine%' THEN 'dobutamine'
        END as vasopressor_type
    FROM `sccm-discovery.eicu_crd_ii_v0_2_0.infusiondrug` i
    WHERE (LOWER(i.drugname) LIKE '%norepinephrine%' OR LOWER(i.drugname) LIKE '%noradrenaline%'
           OR LOWER(i.drugname) LIKE '%epinephrine%' OR LOWER(i.drugname) LIKE '%adrenaline%'
           OR LOWER(i.drugname) LIKE '%dopamine%' OR LOWER(i.drugname) LIKE '%vasopressin%'
           OR LOWER(i.drugname) LIKE '%phenylephrine%' OR LOWER(i.drugname) LIKE '%dobutamine%')
        AND SAFE_CAST(i.drugrate AS FLOAT64) > 0
        AND i.infusionoffset IS NOT NULL
)
SELECT * FROM medication_vaso
UNION ALL
SELECT * FROM infusion_vaso;


"""
vasopressin_admin = client.query(query).to_dataframe()
vasopressin_admin

In [ ]:
query = f"""

  SELECT distinct
    p.patientunitstayid,
    p.patienthealthsystemstayid,
    p.hospitalid,
    p.age,
    p.gender,
    p.ethnicity,
    p.hospitaldischargeoffset,
    p.unitdischargeoffset,
    p.unittype,
    CASE
      WHEN p.age = '> 89' THEN 90
      WHEN SAFE_CAST(p.age AS INT64) IS NOT NULL THEN SAFE_CAST(p.age AS INT64)
      ELSE NULL
    END as age_numeric
  FROM `sccm-discovery.eicu_crd_ii_v0_2_0.patient` p
  WHERE p.age != '' AND p.age IS NOT NULL


"""
patient_info = client.query(query).to_dataframe()
patient_info

In [ ]:
query = f"""


"""
client.query(query).to_dataframe()

In [ ]:
query = f"""


"""
client.query(query).to_dataframe()

In [ ]:
query = f"""


"""
client.query(query).to_dataframe()